#### F vector

consider this sample is our router probability for all 5 tokens and 4 experts (5x4) matrix

In [56]:
sample = torch.randn(5,4)
sample

tensor([[ 0.0384,  0.3811, -0.9004,  0.0853],
        [ 0.2770,  0.1141, -0.6625,  0.4889],
        [ 0.7854,  0.7123, -0.3660, -1.2273],
        [ 0.9355,  1.9071,  0.7386, -0.3621],
        [ 0.8633, -0.5028, -1.0617, -1.2414]])

In [57]:
val,ind = torch.topk(sample,k=1,dim=-1)
val,ind

(tensor([[0.3811],
         [0.4889],
         [0.7854],
         [1.9071],
         [0.8633]]),
 tensor([[1],
         [3],
         [0],
         [1],
         [0]]))

In [58]:
f_vector = torch.zeros_like(sample)
ones = torch.ones_like(sample)

f_vector

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [59]:
f_vector.scatter_(dim=-1,index=ind, src=ones)

tensor([[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.]])

In [60]:
f_vector.sum(dim=0)

tensor([2., 2., 0., 1.])

In [61]:
f_vector = f_vector.sum(dim=0)/f_matrix.shape[0]
f_vector

tensor([0.4000, 0.4000, 0.0000, 0.2000])

see the imbalance? each expert should get 1/4= 0.25 tokens, we need to minimize this inappropriate distribution by adding this in loss function.

#### P vector

In [62]:
sample

tensor([[ 0.0384,  0.3811, -0.9004,  0.0853],
        [ 0.2770,  0.1141, -0.6625,  0.4889],
        [ 0.7854,  0.7123, -0.3660, -1.2273],
        [ 0.9355,  1.9071,  0.7386, -0.3621],
        [ 0.8633, -0.5028, -1.0617, -1.2414]])

In [63]:
sample = sample.softmax(dim=-1)
sample

tensor([[0.2599, 0.3661, 0.1016, 0.2724],
        [0.2877, 0.2444, 0.1124, 0.3555],
        [0.4203, 0.3907, 0.1329, 0.0562],
        [0.2111, 0.5578, 0.1734, 0.0577],
        [0.6567, 0.1675, 0.0958, 0.0800]])

In [64]:
p_vector = sample.sum(dim=0)/sample.shape[0]
p_vector

tensor([0.3671, 0.3453, 0.1232, 0.1644])

as you can see the experts have imbalanced distribution of probabilities, the objective of including P vector is to make this p vector's distribution uniform

#### loss function

In [69]:
N = 4
alpha = 1.0

In [71]:
loss = alpha * N * (p_vector*f_vector).sum()
loss

tensor(1.2714)